In [1]:
import pymongo
import os
import json
from dotenv import load_dotenv
import torch
load_dotenv("../gorilla_openfunctions/.env")

True

In [2]:
_URI = f"mongodb://{os.environ['MONGODB_USERNAME']}:{os.environ['MONGODB_PASSWORD']}@{os.environ['MONGODB_HOST']}:{os.environ['MONGODB_PORT']}/{os.environ['MONGODB_DATABASE']}"
mogodb_client = pymongo.MongoClient(_URI)
raw_functions = mogodb_client["function_calling"]["raw_functions"]

In [3]:
function_description = []
function_embeddings = []
for data in raw_functions.find():
    function_description.append(json.dumps(data["function"]))
    function_embeddings.append(data["openai_embedding"])

ServerSelectionTimeoutError: localhost:27017: [Errno 54] Connection reset by peer (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 6588e4085dd6c773ef3387bc, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 54] Connection reset by peer (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>

In [ ]:
import pandas as pd
df = pd.DataFrame({"function_name": function_description,
                  "function_embeddings": function_embeddings})
df.head()

,function_name,function_embeddings
0,"{""name"": ""Coffee Shop Locator"", ""api_call"": ""c...","[0.01185300387442112, 0.012052271515130997, -0..."
1,"{""name"": ""Flight Booking"", ""api_call"": ""flight...","[0.0012475474504753947, -0.008101481944322586,..."
2,"{""name"": ""Restaurant Reservation"", ""api_call"":...","[0.013252653181552887, -0.00012823790893889964..."
3,"{""name"": ""Weather Forecast"", ""api_call"": ""weat...","[0.007183193229138851, 0.003343613352626562, -..."
4,"{""name"": ""Pharmacy Locator"", ""api_call"": ""phar...","[-0.0023136872332543135, 0.024875197559595108,..."


In [ ]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter("./logs/embeddings")

embeddings = df["function_embeddings"]
embeddings = torch.tensor(embeddings, requires_grad=False)
writer.add_embedding(
    embeddings, metadata=df["function_name"].tolist(), tag='embeddings')